# Hyperparameter Tuning Lab

### Lesson Objectives:
- Understand the difference between parameters and hyperparameters
- Understand why we tune hyperparameters
- Understand how hyperparameter tuning impacts the outcomes of your model
- Apply hyperparameter tuning to your model using gridsearch and randomsearch
- Select the "best" hyperparameters for your model

## Parameters vs Hyperparameters

We talk a lot about parameters and hyperparameters, but what are they?  What is the difference?

![](images/meetupconsumercreditdefaultvers2all-11-638.jpg)

<font size="5"><span style="color:red"> Why do we tune hyperparameters?</span></font>

We tune hyperparameters to find the set of hyperparameters that optimize the predictive ability (ex. accuracy, recall, r-squared value, RMSE, etc.)

A big factor in whether a machine learning model will perform well is a lot of tweaking...

![Pile of data to stir (https://xkcd.com/1838/)](images/machine_learning_xkcd.png)

You can think of hyperparameters as little dials to adjust to make it easier for the machine learning model to learn

![](images/dials.png)

But how do we know what to adjust them to?!

## How does this fit into the CRISP-DM Model?

Let's remind ourselves about the steps of the CRISP-DM Model.

![](./images/new_crisp-dm.png)

## First let's talk data!

The [data](https://www.kaggle.com/jolasa/bay-area-bike-sharing-trips) we will be using today is taken from Kaggle. This dataset contains anonymized trips data of Lyft bike sharing system(Bay Wheels), in the Bay Area from January 2019 to May 2019.


We are going to use this data to create a model the predicts if the the member is a "brogrammer".  Everyone who is a male and lives in SF AND rides an electric bike is a Brogrammer.  This target has already been created for you.

![bikes](images/bikes.jpg) 
![the wiggle](images/wiggle.png)

### Let's read in the data and do some EDA


In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


from sklearn.preprocessing import LabelEncoder

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ClassificationReport

In [ ]:
bike=pd.read_pickle('./data/bike.pkl')
bike.head()

In [ ]:
bike.bg.value_counts()

In [ ]:
bike.describe()

In [ ]:
bike.bg.value_counts().plot(kind='bar')

In [ ]:
bike.user_type_Customer.value_counts().plot(kind='bar')

In [ ]:
bike.user_type_Subscriber.value_counts().plot(kind='bar')

In [ ]:
bike.trip_duration_min.plot(kind='hist')

In [ ]:
bike.member_birth_year.plot(kind='hist')

<font size="5"><span style="color:red"> What other visualizations could/should we create?</span></font>

## Start with a Baseline/Dummy Model

First we are going to start with a dummy model to predict if the user was a brogrammer.  In our dummy model we classify everything as the majority class.  
https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [ ]:
#set random state for our notebook
import numpy as np
np.random.seed(107)

#set brogrammer as the target and use all other features as 
y = bike['bg']
#set features to all variables except brogrammer and trip in seconds
X = bike.drop(['bg', 'trip_duration_sec'], axis=1)

In [ ]:
#split our data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3 )

In [ ]:
#check out the shape of our data
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# DummyClassifier to predict only target 0
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy.fit(X_train, y_train)
dummy.score(X_train, y_train)

<font size="5"><span style="color:red"> What does the "score" from the dummy classifier tell us about the predictive quality of our model?</span></font>

The score from the classifier tells us that 75% of our test data was correctly classified.

#### Now  let's create a nice looking confusion matrix using the yellowbrick package
[yellowbrick documentation](https://www.scikit-yb.org/en/latest/api/classifier/confusion_matrix.html)

In [ ]:
cm = ConfusionMatrix(dummy)

# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_train, y_train)


<font size="5"><span style="color:red"> What does this confusion matrix tell us about what the dummy classifier did?</span></font>

The dummy classifier classified all test data as brogrammer.  

## Your Turn: Fitting a Vanilla Model

Now that we know what our dummy/baseline classifier does we are going to fit a vanilla decision tree model to predict if the user was a brogrammer.

Do the following in your group then answer the questions:

1. Create a vanilla decision tree model and fit it to your training data
2. Print the accuracy score of this prediction
3. Print a confusion matrix for your training data
4. Print a classification report for the model

**BONUS**:  Repeat the above steps using a Naive Bayes classifier. Compare and contrast the two models.

In [ ]:
# your code here

<font size="5"><span style="color:red"> 1. What does the "score" from the decision tree classifier tell us about the predictive quality of our model?</span></font>

**Your answer here**

<font size="5"><span style="color:red"> 2. According to confusion matrix, how well did our decision tree classifier do?</span></font>

**Your answer here**

<font size="5"><span style="color:red">3. According to classification report, how well did our decision tree classifier do?</span></font>

**Your answer here**

## Now onto Grid Search: Find the best hyperparameters for us!

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

A way for us to search over multiple hyperparameters for the given model to see if we can increase the accuracy of our model by "tuning" our hyperparameters in our decision tree model.

In [ ]:
#first we set up a grid of hyperparameters we want to "search over"
param_grid = {
    'max_depth': [2, 5],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
}

#then we conduct the "search"
gs = GridSearchCV(estimator = clf_DT, param_grid = param_grid, 
                          cv = 5)
gs.fit(X_train, y_train)
#finally we find the best hyperparameters
gs.best_params_

###  WOW that took a while!!  Is there an easier way than just waiting for this to run?

## Luckily there is!  Let's check out the RandomizedSearchCV feature in sklearn
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html


In [ ]:
#we can use the same exact param_grid we have already defined
param_grid = {
    'max_depth': [2, 5],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
}

#then we conduct the "search"
rs = RandomizedSearchCV(estimator = clf_DT,  param_distributions = param_grid, 
                          cv = 5)
rs.fit(X_train, y_train)
#finally we find the best hyperparameters
rs.best_params_

### Great, that ran a bit faster.  But wait...why are our best params different now????

In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter.

## Fitting our Decision Tree Using Best Hyperparameters

In [ ]:
#save our best model from the grid search
best_DT=gs.best_estimator_

#fit the data using the best model and look at the score given test data
best_DT.fit(X_train, y_train)
best_DT.score(X_test, y_test)

<font size="5"><span style="color:red"> Our vanilla model had an accuracy score of 76%.  How did our tuned model do in comparison?</span></font>

Looks like our tuned model actually did slightly worse than our vanilla model in overall accuracy.  Maybe it helped though with the accuracy of nice people?

In [ ]:
cm = ConfusionMatrix(best_DT)

# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)


<font size="5"><span style="color:red"> According to confusion matrix, how well did our decision tree classifier do?</span></font>

Wow, this confusion matrix is the same as our dummy classifier! We definitely did better at predicting brogrammers but not well at all in nice people.

In [ ]:
y_pred=best_DT.predict(X_test)
print(classification_report(y_test, y_pred))

<font size="5"><span style="color:red"> According to classification report, how well did our decision tree classifier do?</span></font>

The classification report confirms that our model increased the f1 score a little bit and the recall a lot for the brogrammers but did MUCH worse for identifying our nice people.

## Pipelines and hyperparameter searching!

Yes, let's go back to that concept of a pipeline.  We established using pipelines is important when we our transforming our data and using cross validation (which our gridsearch and randomized search are using).  

In [ ]:
#set up our pipe, even though ss isn't necessary here let's add it 
#just to see how the pipe works
pipe = make_pipeline(StandardScaler(), DecisionTreeClassifier())


#we can use the same exact param_grid we have already defined
param_grid = {
    'decisiontreeclassifier__max_depth': [2, 5],
    'decisiontreeclassifier__max_features': [2, 3],
    'decisiontreeclassifier__min_samples_leaf': [3, 4],
    'decisiontreeclassifier__min_samples_split': [8, 10],
}

#now use our pipe in our grid search
gridsearch_pipe = GridSearchCV(pipe, 
                          param_grid = param_grid,
                          cv = 5)

gridsearch_pipe.fit(X_train, y_train)

In [ ]:
#save our best model from the grid search
best_pipe=gridsearch_pipe.best_estimator_

#fit the data using the best model and look at the score given test data
best_pipe.fit(X_train, y_train)
best_pipe.score(X_test, y_test)

In [ ]:
cm = ConfusionMatrix(best_pipe)

# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)


<font size="5"><span style="color:red"> Does this mean that hyperparameter tuning isn't good?  What can we do now?</span></font>

We have lots of options to move forward and increase our classification.  We can:
- **try a different classification model**
- **tune other hyperparameters in our decision tree model**
- **try balancing our classes**
- add more data
- add new features- maybe we don't have ones that are very predictive

<font size="5"><span style="color:red"> Your Turn:  Now work in small groups to take one of the above next steps to increase the classification of brogrammers! </span></font>